In [1]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup

url = 'https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL'
page = requests.get(url)
soup = BeautifulSoup(page.content, "lxml")

In [2]:
script = soup.find('script', text=re.compile('root\.App\.main'))
script

<script>
(function (root) {
/* -- Data -- */
root.App || (root.App = {});
root.App.now = 1624977446006;
root.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClasses":"Bgc($lv2BgColor)!","Header":{"isFixed":true,"uhContainerClasses":"Bgi($uhGrayGradient) D(n)--print","navContainerClasses":"Bgi($navrailGrayGradient) Bxsh($navrailShadow) Pos(r) hasScrolled_Bxsh(headerShadow) Panel-open_Bxsh(headerShadow)","navTransitionClasses":"HideNavrail_Translate3d(0,-46px,0) Panel-open_Translate3d(0,-46px,0)","secondaryNavContainerClasses":"hasScrolled_Bdbw(0px) Bgc($lv3BgColor) Bxsh($navrailShadow) D(n)--print","height":135},"fetchNewAttribution":true},"meta"

In [3]:
json_text = re.search(r'^\s*root\.App\.main\s*=\s*({.*?})\s*;\s*$',
                      script.string, flags=re.MULTILINE).group(1)
data = json.loads(json_text)
data

{'context': {'dispatcher': {'stores': {'AdStore': {'_childCompositeReady': {},
     '_newPageRendered': False,
     'adBoostGroupConf': {'LREC3': None,
      'LREC3-9': {'extrapositions': ['MON2-9'], 'inview': 'LREC4-9'},
      'LREC4': None,
      'LREC4-9': {'extrapositions': ['MON2-9'], 'inview': 'LREC3-9'},
      'MON2': {'extrapositions': ['LREC3', 'LREC4']},
      'MON2-9': {'extrapositions': ['LREC3-9', 'LREC4-9']}},
     'adFetchEvent': {'launchViewportAdSlot': [],
      'optionalps': '',
      'ps': 'FB2A,FB2B,FB2C,FB2D,MON,LDRB,LREC,LREC2,FOOT,TRADENOW',
      'sa': ' ticker=AAPL Y-BUCKET=finance-US-en-US-def LREC=300x250,1x1 rs="pt:utility;site:finance;ver:ydotcom;pct:qsp;lu:0" migration=1',
      'site': 'finance',
      'sp': 95993639},
     'adblockPos': True,
     'adfetchTimeout': 500,
     'adsMeta': None,
     'allowHostUrlAsReferrer': True,
     'appConfig': {'base': {'aboveFoldPositions': 'WPS,MAST,LDRB,SPRZ,SPL,LREC,BTN,BTN-1,BTN-2,BTN-3,BTN1,BTNA,BTNB,BTNC,BTND,FB

In [4]:
upgrades_and_downgrades = data['context']['dispatcher']['stores']['QuoteSummaryStore']['upgradeDowngradeHistory']['history']
df = pd.DataFrame(upgrades_and_downgrades)

In [5]:
df

,epochGradeDate,firm,toGrade,fromGrade,action
0,1624534652,Morgan Stanley,Overweight,,main
1,1622201116,New Street Research,Sell,Neutral,down
2,1621421955,Barclays,Equal-Weight,,main
3,1619704919,Barclays,Equal-Weight,,main
4,1619703632,"Monness, Crespi, Hardt",Buy,,main
...,...,...,...,...,...
787,1331713260,Canaccord Genuity,Buy,,main
788,1331705580,Morgan Stanley,Overweight,,main
789,1331618880,Jefferies,Buy,,main
790,1331191980,FBN Securities,Outperform,,main


In [6]:
df['date'] = pd.to_datetime(df['epochGradeDate'], unit='s')
df = df.drop(columns=['epochGradeDate'])

In [7]:
df

,firm,toGrade,fromGrade,action,date
0,Morgan Stanley,Overweight,,main,2021-06-24 11:37:32
1,New Street Research,Sell,Neutral,down,2021-05-28 11:25:16
2,Barclays,Equal-Weight,,main,2021-05-19 10:59:15
3,Barclays,Equal-Weight,,main,2021-04-29 14:01:59
4,"Monness, Crespi, Hardt",Buy,,main,2021-04-29 13:40:32
...,...,...,...,...,...
787,Canaccord Genuity,Buy,,main,2012-03-14 08:21:00
788,Morgan Stanley,Overweight,,main,2012-03-14 06:13:00
789,Jefferies,Buy,,main,2012-03-13 06:08:00
790,FBN Securities,Outperform,,main,2012-03-08 07:33:00


In [8]:
action_map = {
    'init': 'initiated',
    'down': 'downgrade',
    'reit': 'reiterates',
    'up': 'upgrade',
    'main': 'maintains'
}


for key, value in action_map.items():
    df.replace(key, value, inplace=True)

In [9]:
df

,firm,toGrade,fromGrade,action,date
0,Morgan Stanley,Overweight,,maintains,2021-06-24 11:37:32
1,New Street Research,Sell,Neutral,downgrade,2021-05-28 11:25:16
2,Barclays,Equal-Weight,,maintains,2021-05-19 10:59:15
3,Barclays,Equal-Weight,,maintains,2021-04-29 14:01:59
4,"Monness, Crespi, Hardt",Buy,,maintains,2021-04-29 13:40:32
...,...,...,...,...,...
787,Canaccord Genuity,Buy,,maintains,2012-03-14 08:21:00
788,Morgan Stanley,Overweight,,maintains,2012-03-14 06:13:00
789,Jefferies,Buy,,maintains,2012-03-13 06:08:00
790,FBN Securities,Outperform,,maintains,2012-03-08 07:33:00


In [10]:
df = df[['date', 'firm', 'action', 'fromGrade', 'toGrade']]

In [11]:
df

,date,firm,action,fromGrade,toGrade
0,2021-06-24 11:37:32,Morgan Stanley,maintains,,Overweight
1,2021-05-28 11:25:16,New Street Research,downgrade,Neutral,Sell
2,2021-05-19 10:59:15,Barclays,maintains,,Equal-Weight
3,2021-04-29 14:01:59,Barclays,maintains,,Equal-Weight
4,2021-04-29 13:40:32,"Monness, Crespi, Hardt",maintains,,Buy
...,...,...,...,...,...
787,2012-03-14 08:21:00,Canaccord Genuity,maintains,,Buy
788,2012-03-14 06:13:00,Morgan Stanley,maintains,,Overweight
789,2012-03-13 06:08:00,Jefferies,maintains,,Buy
790,2012-03-08 07:33:00,FBN Securities,maintains,,Outperform


In [12]:
df.to_excel('aapl-upgrades-downgrades.xlsx')

In [13]:
df.shape

(792, 5)

In [14]:
df.columns

Index(['date', 'firm', 'action', 'fromGrade', 'toGrade'], dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       792 non-null    datetime64[ns]
 1   firm       792 non-null    object        
 2   action     792 non-null    object        
 3   fromGrade  792 non-null    object        
 4   toGrade    792 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 31.1+ KB


In [17]:
df.firm.value_counts()

Morgan Stanley         53
Barclays               39
Canaccord Genuity      39
UBS                    34
Nomura                 32
                       ..
Morgan Keegan           1
Phillip Securities      1
Oxen Group              1
New Street Research     1
Sterne Agee CRT         1
Name: firm, Length: 93, dtype: int64

In [18]:
df.action.value_counts()

maintains     613
downgrade      63
upgrade        49
initiated      44
reiterates     23
Name: action, dtype: int64

In [20]:
df.fromGrade.value_counts()

                  602
Buy                61
Outperform         29
Neutral            23
Overweight         23
Hold               16
Market Perform     16
Equal-Weight        5
Strong Buy          4
Long-Term Buy       3
Sell                3
Sector Weight       2
Accumulate          1
Perform             1
Reduce              1
Equal-weight        1
Sector Perform      1
Name: fromGrade, dtype: int64

In [21]:
df.toGrade.value_counts()

Buy                  306
Outperform           158
Overweight           122
Neutral               81
Hold                  39
Market Perform        24
Equal-Weight          19
Sell                   8
Strong Buy             8
Positive               4
Sector Perform         4
                       3
Market Outperform      3
Perform                2
Sector Outperform      2
Sector Weight          2
Long-Term Buy          1
Long-term Buy          1
Underweight            1
Negative               1
Equal-weight           1
Reduce                 1
Underperform           1
Name: toGrade, dtype: int64